In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from tmp0206_benchmark_highfold import place_holder_init,run
import pandas as pd
from tqdm import tqdm
from numpy import vectorize
import glob
from tmp0121_CycBc import init_task,bindcraft_score,BasicDict,mk_afdesign_model
import numpy as np
from colabdesign.af.alphafold.common import residue_constants
import os
import time
from typing import Dict,List
from pathlib import Path
aa_order = residue_constants.restype_order
order_aa = {b:a for a,b in aa_order.items()}


    
def place_holder_init(design_path:str,**kwargs):
    '''
    creat the file system, init some global variables.
    kwargs are used to override `advanced_settings_path`
    '''

    starting_pdb = 'Holder-PDB'
    chains = "Holder-Chain"
    binder_name = "Holder-Name" # an arbitrary name
    (target_settings, advanced_settings, filters,
        settings_file,filters_file,advanced_file,
        design_models, prediction_models, multimer_validation,
        design_paths,trajectory_csv,mpnn_csv,final_csv,failure_csv,
        trajectory_dirs
        )=init_task(
        design_path = design_path,
        binder_name = binder_name, # an arbitrary name
        starting_pdb = starting_pdb,
        chains = chains ,
        ### NO USE ###
        load_previous_target_settings="",
        advanced_settings_path='settings_advanced/default_4stage_multimer_cyc.json',
        filter_settings_path='settings_filters/no_filters.json',
        target_hotspot_residues = "",
        lengths = "10,20",
        number_of_final_designs = 1,
        )
    advanced_settings.update(kwargs)
    return (target_settings, advanced_settings, filters,
        settings_file,filters_file,advanced_file,
        design_models, prediction_models, multimer_validation,
        design_paths,trajectory_csv,mpnn_csv,final_csv,failure_csv,
        trajectory_dirs
        )

def run(
    # basic
    starting_pdb:os.PathLike,
    chains:str,
    binder_name:str,
    binder_seq:str,
    # from place_holder_init, change when you know what you are doing
    target_settings:Dict[str,str|int|List[int]],
    advanced_settings:BasicDict,
    filters:Dict[str,Dict[str,float|bool|None]],
    prediction_models:List[int],
    design_paths:Dict[str,str],
    # for rerun
    complex_prediction_model:mk_afdesign_model=None,
    binder_prediction_model:mk_afdesign_model=None,
    ):
    starting_pdb=Path(starting_pdb)
    target_settings['binder_name']=starting_pdb.stem
    target_settings['starting_pdb']=str(starting_pdb.absolute())
    target_settings['chains']=chains
    return bindcraft_score(
        binder_seq,
        design_name=binder_name,
        target_settings=target_settings,
        advanced_settings=advanced_settings,
        filters=filters,
        prediction_models=prediction_models,
        design_paths=design_paths,
        trajectory_pdb= None,
        )
    


In [3]:
score_dir='output/score_dir'
print("run this to construct file system, create global variables & start PyRosetta")
(target_settings, advanced_settings, filters,
    settings_file,filters_file,advanced_file,
    design_models, prediction_models, multimer_validation,
    design_paths,trajectory_csv,mpnn_csv,final_csv,failure_csv,
    trajectory_dirs
    )=place_holder_init(score_dir)

run this to construct file system, create global variables & start PyRosetta
┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.conda.ubuntu.cxx11thread.serialization.Ubuntu.python310.Release 2025.03+release.1f5080a079a5261122c0e532c46f61a4f7e20df8 2025-01-13T15:37:50] retrieved from: http://www.pyrosetta.org


In [3]:
#Don't run this if you haven't install pymol
from pymol import cmd
from pymol import stored
def grab_pdb_info()->Dict[str,str]:
    pdb_info={}
    for f in glob.glob(f'input/HighFold_data/mul_baseline_native/*.pdb'):
        cmd.load(f)
        for x in cmd.get_names():
            # print(x)
            pdb_info[x]={'target':[]}
            for i in cmd.get_chains(x):
                cmd.select(f'chain_{i}',f'chain {i} and name CA and alt A+')
                stored.s=0
                stored.binder=[]
                def _(atom):
                    stored.s+=1
                cmd.iterate(f'chain_{i}',_)
                # print(f'{i}: {stored.s}')
                if stored.s==0:
                    # pdb_info[x][i]=stored.s
                    pass
                elif stored.s>100:
                    pdb_info[x]['target'].append(i)
                else:
                    cmd.iterate(f'chain_{i}',lambda atom:stored.binder.append(atom.oneletter))
                    pdb_info[x]['binder']=''.join(stored.binder)
                    pdb_info[x]['binder_chain']=i
            pdb_info[x]['target']=','.join(pdb_info[x]['target'])
            cmd.delete(x)
        return pdb_info

In [4]:
df:pd.DataFrame=pd.read_csv('input/RFD-pep-benchmark/RbtA.csv')
df=df.sort_values(by='Seq',key=vectorize(lambda x:len(x)),ascending=True,ignore_index=True)
starting_pdb='input/RFD-pep-benchmark/AF-A0A7Z1WTP0-F1-model_v4-trunc.pdb'
chains='A'
print('now you only need following args to run scoring:\n')
binder_name,binder_seq=df['ID'].iloc[0],df['Seq'].iloc[0]
starting_pdb,chains,
print(f'{starting_pdb=}\n{chains=}\n{binder_name=}\n{chains=}')

now you only need following args to run scoring:

starting_pdb='input/RFD-pep-benchmark/AF-A0A7Z1WTP0-F1-model_v4-trunc.pdb'
chains='A'
binder_name='RBB_D1'
chains='A'


In [7]:
print('run batch of scoring as follow. \nSort your query by `input_pdb_file` & `binder_seq_length` first, so that folding model could be reused!')

run batch of scoring as follow. 
Sort your query by `input_pdb_file` & `binder_seq_length` first, so that folding model could be reused!


In [ ]:
binder_prediction_model,complex_prediction_model=None,None
last_len=0
for name,seq in tqdm(zip(df['ID'],df['Seq'])):
    if len(seq)!=last_len:
        binder_prediction_model,complex_prediction_model=None,None
    last_len=len(seq)
    
    (mpnn_complex_averages,binder_averages,seq_notes,
    complex_prediction_model,binder_prediction_model)=run(
        starting_pdb=starting_pdb,
        chains=chains,
        binder_name=name,
        binder_seq=seq,
        target_settings=target_settings,
        advanced_settings=advanced_settings,
        filters=filters,
        prediction_models=[0,1,2,3,4],
        design_paths=design_paths,
        binder_prediction_model=binder_prediction_model,
        complex_prediction_model=complex_prediction_model
    )
    complex_prediction_model.restart()
    binder_prediction_model.restart()

5it [1:01:51, 742.32s/it]
